# Data engineer project

## 0. Libraries

In [1]:
import pandas as pd
import psycopg2

## 1. Functions

In [2]:
def create_db():
    """
        This function create a database a setup a connection.
        
        Returns:
            conn: psycopg2 connection object
            cur: cursor
    """
    # connect to default database
    # example: conn = psycopg2.connect("host=localhost dbname=bd user=ljpcastroc password=password")
    conn = psycopg2.connect("host=localhost dbname=dmodeldb user=ljpcastroc password=password")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP   DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")
    # close connection to default database
    conn. close()
    conn = psycopg2.connect("host=localhost dbname=accounts user=ljpcastroc password=password")
    # we'll disable this option
    #conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    """
        drop table with a query
        
        return: nothing
    """
    for query in drop_table_queries:
        cur.execute(query)
        cur.commit()

## 2. Data exploration

### 2.1 Country dimension

We'll select all columns.

In [7]:
acc_country = pd.read_csv("data/dimension_country.csv")
acc_country.head()

,Country Code,Country Name
0,AFE,Africa Eastern and Southern
1,AFW,Africa Western and Central
2,ARB,Arab World
3,CSS,Caribbean small states
4,CEB,Central Europe and the Baltics


In [11]:
acc_country.columns

Index(['Country Code', 'Country Name'], dtype='object')

In [17]:
acc_country = acc_country.rename(columns={'Country Code': 'country_code',
                                          'Country Name': 'country_name'})

In [18]:
for i in acc_country.columns:
    print(i, ": ", acc_country[i].str.len().max())

country_code :  3
country_name :  52


### 2.2 Indicator code dimension

We'll select all columns.

In [8]:
acc_indicat = pd.read_csv("data/dimension_indicator.csv")
acc_indicat.head()

,Indicator Code,Indicator Name
0,IC.BUS.DISC.XQ,Business extent of disclosure index (0=less di...
1,IC.CRD.INFO.XQ,Depth of credit information index (0=low to 8=...
2,FS.AST.PRVT.GD.ZS,Domestic credit to private sector (% of GDP)
3,EG.USE.ELEC.KH.PC,Electric power consumption (kWh per capita)
4,EG.IMP.CONS.ZS,"Energy imports, net (% of energy use)"


In [13]:
acc_indicat.columns

Index(['Indicator Code', 'Indicator Name'], dtype='object')

In [20]:
acc_indicat = acc_indicat.rename(columns={'Indicator Code': 'indicator_code',
                                          'Indicator Name': 'indicator_name'})

In [23]:
for i in acc_indicat.columns:
    print(i, ": ", acc_indicat[i].str.len().max())

indicator_code :  17
indicator_name :  96


### 2.3 Fac table

We'll select all columns.

In [10]:
acc_fact = pd.read_csv("data/facttable.csv")
acc_fact.head()

,Country Code,Indicator Code,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AFE,IC.BUS.DISC.XQ,NaN,NaN,NaN,NaN,NaN,3.730769,3.730769,3.730769,...,4.230769,4.346154,4.423077,4.538462,4.692308,4.730769,4.884615,4.884615,NaN,NaN
1,AFE,IC.CRD.INFO.XQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.240000,2.400000,3.269231,3.769231,4.346154,4.423077,4.500000,NaN,NaN
2,AFE,FS.AST.PRVT.GD.ZS,74.979893,77.003130,62.432376,71.326432,80.268045,85.821854,94.990002,94.000221,...,77.608895,74.179365,72.396334,65.157383,62.907686,64.246009,65.290639,63.562336,59.744225,NaN
3,AFE,EG.USE.ELEC.KH.PC,780.702624,743.916044,769.080854,774.334166,777.672091,780.851052,789.487840,796.901936,...,703.908401,698.369914,686.728057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFE,EG.IMP.CONS.ZS,-31.391070,-29.136323,-32.910884,-32.419994,-31.161816,-35.939423,-38.805789,-41.694275,...,-32.244992,-32.035007,-31.655426,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
acc_fact.columns

Index(['Country Code', 'Indicator Code', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
      dtype='object')

## 3. DB Designer